In [1]:
import pandas as pd
import numpy as np

# Modelo Recurrentes con Embeddings a nivel de caracter

In [2]:
df = pd.read_csv('data/acetylcholinesterase_02_bioactivity_data_preprocessed.csv')

In [3]:
df.head()

,molecule_chembl_id,canonical_smiles,standard_value,standard_value_norm,pIC50
0,CHEMBL133897,CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1,750.0,750.0,6.124939
1,CHEMBL336398,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1,100.0,100.0,7.000000
2,CHEMBL131588,CN(C(=O)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F)c1ccccc1,50000.0,50000.0,4.301030
3,CHEMBL130628,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F,300.0,300.0,6.522879
4,CHEMBL130478,CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N(C)C,800.0,800.0,6.096910


# Canonical_smiles To secuence

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2023-08-31 06:32:26.813894: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-31 06:32:26.817384: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-31 06:32:26.905278: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-31 06:32:26.908402: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-31 06:32:27.891046: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Utilizando Tokenizer y pad_sequences de keras a nivel de caracter, obtener la entrada tokenizada. Por ejemplo:

**El smiles**:

'C[C@@]12CC[C@H]3[C@]4(C)CCC[C@@]5(C)C(=O)OCC[N+](C)(C)CCCCCCCCC[N+](C)(C)CCOC(=O)[C@]6(C)CCC[C@]7(C)[C@@H]6CC[C@@]68C[C@@H](OC(=O)CCCCCCCCC(=O)O[C@@H]1C[C@@]3(CC[C@@H]45)C2)[C@@](C)(CC[C@@H]76)C8.[Br-].[Br-]'

**Tendría que quedar algo asi (puede que no sea exacto):**

array([ 2, 11,  2, 13, 13, 12,  5,  7,  2,  2, 11,  2, 13, 16, 12, 10, 11,
        2, 13, 12, 15,  3,  2,  4,  2,  2,  2, 11,  2, 13, 13, 12, 23,  3,
        2,  4,  2,  3,  9,  6,  4,  6,  2,  2, 11,  8, 21, 12,  3,  2,  4,
        3,  2,  4,  2,  2,  2,  2,  2,  2,  2,  2,  2, 11,  8, 21, 12,  3,
        2,  4,  3,  2,  4,  2,  2,  6,  2,  3,  9,  6,  4, 11,  2, 13, 12,
       30,  3,  2,  4,  2,  2,  2, 11,  2, 13, 12, 35,  3,  2,  4, 11,  2,
       13, 13, 16, 12, 30,  2,  2, 11,  2, 13, 13, 12, 30, 36,  2, 11,  2,
       13, 13, 16, 12,  3,  6,  2,  3,  9,  6,  4,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  3,  9,  6,  4,  6, 11,  2, 13, 13, 16, 12,  5,  2, 11,
        2, 13, 13, 12, 10,  3,  2,  2, 11,  2, 13, 13, 16, 12, 15, 23,  4,
        2,  7,  4, 11,  2, 13, 13, 12,  3,  2,  4,  3,  2,  2, 11,  2, 13,
       13, 16, 12, 35, 30,  4,  2, 36, 20, 11, 25, 26, 17, 12, 20, 11, 25,
       26, 17, 12], dtype=int32)
       

**Otro ejemplo:**
smiles: 'NC(=O)O'

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 8, 2, 3, 9, 6, 4, 6], dtype=int32)

**Tener en cuenta:**

- Ver cuanta es la máxima longitud para agregar el padding correspondiente
- En el ejemplo se agrego padding a la izquierda

In [5]:
max_len_idx = df['canonical_smiles'].apply(len).argmax()
min_len_idx = df['canonical_smiles'].apply(len).argmin()

In [6]:
min_sequence_length = df['canonical_smiles'].apply(len).min()
max_sequence_length = df['canonical_smiles'].apply(len).max()

In [7]:
max_sequence_length

204

In [8]:
X = df['canonical_smiles']
y = df['pIC50']

In [9]:
# Implementar tokenización y guardar en X_seq_pad el dataset tokenizado

In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Extract canonical smiles from the DataFrame and labels
canonical_smiles = X.tolist()
labels = y.tolist()

# Initialize the Tokenizer
tokenizer = Tokenizer(char_level=True)

# Fit the tokenizer on the canonical smiles
tokenizer.fit_on_texts(canonical_smiles)

# Tokenize and pad the canonical smiles sequences
canonical_smiles_tokenized = tokenizer.texts_to_sequences(canonical_smiles)
max_sequence_length = max(map(len, canonical_smiles_tokenized))
canonical_smiles_padded = pad_sequences(canonical_smiles_tokenized, 
                                        maxlen=max_sequence_length)

# Convert labels to a numpy array
labels_array = np.array(labels)

# Convert padded sequences to a numpy array
X_seq_pad = np.array(canonical_smiles_padded)

In [11]:
X_seq_pad.shape[1]

204

In [12]:
# # Find rows with inf values
# rows_with_inf = np.any(np.isinf(X_seq_pad), axis=1)

# # Remove rows with inf values
# X_seq_pad = X_seq_pad[~rows_with_inf]

# X_seq_pad.shape



In [13]:
X_seq_pad[200]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 6, 1, 4, 1, 7, 1, 2, 6, 1, 9, 1, 4, 1, 1, 1, 1, 9,
       3, 1, 1, 1, 1, 7], dtype=int32)

# Split

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
# Convert labels to a numpy array
X_train, X_test, y_train, y_test = train_test_split(X_seq_pad, labels_array, test_size=0.2, random_state=999)
X_train = np.delete(X_train,slice(4100,4150),axis=0)
y_train = np.delete(y_train,slice(4100,4150),axis=0)

In [16]:
len(X_train), len(y_train), len(X_test) , len(y_test)

(4875, 4875, 1232, 1232)

# Network Model

In [17]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional, Dropout, Activation, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import backend as K

In [18]:
# Métrica
def R2(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [19]:
# vocab_size = # Completar largo del vocabulario
vocab_size = len(list(tokenizer.word_index.keys()))

In [20]:
max_sequence_len = len(max(X_seq_pad, key=len))

# Armar modelo en keras de LSTM
El de abajo es un ejemplo propuesto pero puede armar otro similar (siempre con LSTM o GRU)

In [21]:
X_train[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  5,
        1,  2,  8,  5,  3,  1,  4,  8,  1,  2,  1,  3,  5,  1,  7,  6,  1,
        9,  1,  2,  1,  2,  6,  3,  1,  7, 10,  1, 12, 14, 11,  4,  1,  4,
        1,  1,  1,  1,  2,  5,  1,  3,  1,  4,  3,  1,  1,  1,  1,  1,  9],
      dtype=int32)

In [22]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, Dense, BatchNormalization, Activation, LSTM
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Build the sequential model
model = Sequential()

# Embedding layer
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=X_seq_pad.shape[1]))

# Bidirectional LSTM layer with L2 regularization
model.add(Bidirectional(LSTM(125, return_sequences=False, kernel_regularizer=l2(0.01))))

# Dense layer with L2 regularization
model.add(Dense(1000, kernel_regularizer=l2(0.01)))

# Batch normalization layer
model.add(BatchNormalization())

# Activation layer
model.add(Activation('relu'))

# Output dense layer
model.add(Dense(1))

# Print the model summary
model.summary()

# Learning rate scheduler
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001, clipvalue=0.1), loss='mse', metrics=[R2])



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 204, 128)          4352      
                                                                 
 bidirectional (Bidirection  (None, 250)               254000    
 al)                                                             
                                                                 
 dense (Dense)               (None, 1000)              251000    
                                                                 
 batch_normalization (Batch  (None, 1000)              4000      
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 1000)              0         
                                                                 
 dense_1 (Dense)             (None, 1)                 1

In [23]:
mcp = ModelCheckpoint('models/best_model_{epoch}', save_best_only=True, save_format="h5")

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), callbacks=[lr_scheduler, early_stopping,mcp])

Epoch 1/50
77/77 [==============================] - ETA: 0s - loss: 12.9089 - R2: -2.0417INFO:tensorflow:Assets written to: models/best_model_1/assets


INFO:tensorflow:Assets written to: models/best_model_1/assets


77/77 [==============================] - 38s 423ms/step - loss: 12.9089 - R2: -2.0417 - val_loss: 22.8238 - val_R2: -7.3676 - lr: 0.0010
Epoch 2/50
77/77 [==============================] - ETA: 0s - loss: 5.0583 - R2: -0.0770INFO:tensorflow:Assets written to: models/best_model_2/assets


INFO:tensorflow:Assets written to: models/best_model_2/assets


77/77 [==============================] - 31s 409ms/step - loss: 5.0583 - R2: -0.0770 - val_loss: 17.2592 - val_R2: -5.4422 - lr: 0.0010
Epoch 3/50
77/77 [==============================] - 20s 261ms/step - loss: 4.0508 - R2: 0.0504 - val_loss: 20.2986 - val_R2: -6.9690 - lr: 0.0010
Epoch 4/50
77/77 [==============================] - ETA: 0s - loss: 3.4432 - R2: 0.1306INFO:tensorflow:Assets written to: models/best_model_4/assets


...

77/77 [==============================] - 42s 548ms/step - loss: 1.9083 - R2: 0.3196 - val_loss: 2.4671 - val_R2: 0.1133 - lr: 0.0010
Epoch 12/50
77/77 [==============================] - ETA: 0s - loss: 1.8839 - R2: 0.3069INFO:tensorflow:Assets written to: models/best_model_12/assets


INFO:tensorflow:Assets written to: models/best_model_12/assets


77/77 [==============================] - 46s 597ms/step - loss: 1.8839 - R2: 0.3069 - val_loss: 2.0058 - val_R2: 0.2928 - lr: 0.0010
Epoch 13/50
77/77 [==============================] - 26s 341ms/step - loss: 1.8222 - R2: 0.3182 - val_loss: 2.4778 - val_R2: 0.0822 - lr: 0.0010
Epoch 14/50
77/77 [==============================] - 38s 492ms/step - loss: 1.7760 - R2: 0.2949 - val_loss: 2.1160 - val_R2: 0.2052 - lr: 0.0010
Epoch 15/50
77/77 [==============================] - 39s 501ms/step - loss: 1.8636 - R2: 0.2824 - val_loss: 2.9416 - val_R2: -0.1164 - lr: 0.0010
Epoch 16/50
77/77 [==============================] - ETA: 0s - loss: 1.7193 - R2: 0.3347INFO:tensorflow:Assets written to: models/best_model_16/assets


INFO:tensorflow:Assets written to: models/best_model_16/assets


77/77 [==============================] - 54s 701ms/step - loss: 1.7193 - R2: 0.3347 - val_loss: 1.8921 - val_R2: 0.2993 - lr: 0.0010
Epoch 17/50
77/77 [==============================] - 34s 438ms/step - loss: 1.7197 - R2: 0.3320 - val_loss: 2.2852 - val_R2: 0.1559 - lr: 0.0010
Epoch 18/50
77/77 [==============================] - 36s 470ms/step - loss: 1.7576 - R2: 0.3068 - val_loss: 2.5903 - val_R2: 0.0150 - lr: 0.0010
Epoch 19/50
77/77 [==============================] - 38s 489ms/step - loss: 1.6915 - R2: 0.3379 - val_loss: 2.1088 - val_R2: 0.2315 - lr: 0.0010
Epoch 20/50
77/77 [==============================] - 25s 327ms/step - loss: 1.6646 - R2: 0.3428 - val_loss: 2.3696 - val_R2: 0.1098 - lr: 0.0010
Epoch 21/50
77/77 [==============================] - 32s 417ms/step - loss: 1.7012 - R2: 0.3290 - val_loss: 2.0539 - val_R2: 0.2191 - lr: 0.0010
Epoch 22/50
21/77 [=======>......................] - ETA: 13s - loss: 1.5511 - R2: 0.3866

In [ ]:
y_pred = model.predict(X_test)

# Score

In [ ]:
r2 = 1 - ((y_test - y_pred.reshape(-1)) ** 2).sum() / ((y_test - y_test.mean()) ** 2).sum()

In [ ]:
r2
# 0.4984533246797399

In [ ]:
model.save('models/final_v2.h5', )

# Load model (solo si se grabo el modelo)

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model_loaded = load_model('models/final_v1.h5', custom_objects={'R2': R2})

In [ ]:
y_pred = model_loaded.predict(X_test)

¿Hasta que valores de r2 logró?

In [ ]:
r2 = 1 - ((y_test - y_pred.reshape(-1)) ** 2).sum() / ((y_test - y_test.mean()) ** 2).sum()
r2

# Evaluación

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(color_codes=True)
sns.set_style("white")

ax = sns.regplot(x=y_test, y=y_pred, scatter_kws={'alpha':0.4})
ax.set_xlabel('Experimental pIC50', fontsize='large', fontweight='bold')
ax.set_ylabel('Predicted pIC50', fontsize='large', fontweight='bold')
ax.set_xlim(0, 12)
ax.set_ylim(0, 12)
ax.figure.set_size_inches(5, 5)
plt.show